In [30]:
import os
import numpy as np
import librosa

import pandas as pd

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

from tqdm import tqdm
import random
from sklearn.metrics import f1_score
import wandb
from audiomentations import AddBackgroundNoise
from tensorflow.keras.layers import Lambda



In [31]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
class CONFIG:
    SEED = 42
    CLASSIFIER_LR = 3e-4
    EPOCH = 200
    BATCH_SIZE = 8

    SR = 16000
    DURATION = 20 # ms
    FRAME_SIZE = 320

    N_FFT = 64 # (SR*4ms/1000)
    HOP_LENGTH = 32 # (SR*2ms/1000)
    VOICE_DIR = './cv-corpus-19.0-2024-09-13/ko/clips/'
    NOISE_DIR = './ESC-50-master/audio/'

In [33]:
def periodic_hann_window(window_length, dtype):
    return 0.5 - 0.5 * tf.math.cos(2.0 *
                                   np.pi *
                                   tf.range(tf.cast(window_length, tf.float32)) /
                                   tf.cast(window_length, tf.float32))

In [34]:
def wave2log_mel_spectrogram(wave):
    signal_stft = tf.signal.stft(tf.cast(wave, tf.float32),
                                 frame_length=400,
                                 frame_step=160,
                                 fft_length=1024,
                                 window_fn=periodic_hann_window)
    # print(signal_stft.shape) # (98, 513)
    spectogram = tf.abs(signal_stft)

    linear_to_mel = tf.signal.linear_to_mel_weight_matrix(40,
                                signal_stft.shape[-1],
                                16000,
                                300.0,
                                4000.0)
    mel_spectrogram = tf.tensordot(spectogram, linear_to_mel, 1)
    log_mel_spectrogram = tf.math.log(mel_spectrogram + 1e-12)
    return log_mel_spectrogram

In [ ]:
def representative_data_gen():
    valid_df = pd.read_csv('valid_dataset.csv')
    for _, row in valid_df.iterrows():
        path = row['path']
        label = row['label']

        audio_raw = tf.io.read_file(path)
        wave, sr = tf.audio.decode_wav(audio_raw, desired_channels=1)
        wave = tf.squeeze(wave, axis=-1)
        # print(wave.shape) (16000, )
        log_mel_spectrogram = wave2log_mel_spectrogram(wave)
        log_mel_spectrogram = np.expand_dims(log_mel_spectrogram, axis=-1)
        log_mel_spectrogram = np.expand_dims(log_mel_spectrogram, axis=0)

        # print(log_mel_spectrogram.shape) 

        yield [log_mel_spectrogram.astype(np.float32)]


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('./vad_slimnet')
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32


converter.representative_dataset = representative_data_gen
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
converter._experimental_lower_tensor_list_ops = False

converter.experimental_new_converter = True
#converter.allow_custom_ops = False
converter.experimental_enable_resource_variables = True

# 모델 변환
tflite_model = converter.convert()

# TF Select ops: TensorListReserve, TensorListSetItem, TensorListStack 지원하지 않는다
# https://www.tensorflow.org/lite/guide/op_select_allowlist?hl=ko&_gl=1*2zbz7k*_up*MQ..*_ga*MTU4NzE4MDgzMS4xNzMyMjY2NTQx*_ga_W0YLR4190T*MTczMjI2NjU0MS4xLjAuMTczMjI2NjU0MS4wLjAuMA..
# raw ops로는 지원하나, tflite로는 지원하고 있지 않음

# https://www.tensorflow.org/lite/guide/ops_compatibility?hl=ko&_gl=1*1747d60*_up*MQ..*_ga*MTU4NzE4MDgzMS4xNzMyMjY2NTQx*_ga_W0YLR4190T*MTczMjI2NjU0MS4xLjAuMTczMjI2NjY2Mi4wLjAuMA..
# CUSTOM: 다음 TensorFlow Lite 연산도 제공되지만 사용자 정의 모델에 사용할 준비가 되지 않았습니다.
with open('vad_slimnet.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1732266498.191898 2836236 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1732266498.191912 2836236 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2024-11-22 18:08:18.192042: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./vad_slimnet
2024-11-22 18:08:18.192651: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-11-22 18:08:18.192661: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./vad_slimnet
2024-11-22 18:08:18.197256: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-11-22 18:08:18.216252: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./vad_slimnet
2024-11-22 18:08:18.224654: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 32615 microseconds.
loc(callsite(callsite(fused["TensorListReserve:", "fun

ConverterError: Could not translate MLIR to FlatBuffer.<unknown>:0: error: loc(callsite(callsite(fused["TensorListReserve:", "functional_1_1/custom_attention_model_10_1/rnn_12_1/TensorArrayV2_1@__inference___call___1940569"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper___call___1940604"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): 'tf.TensorListReserve' op is neither a custom op nor a flex op
<unknown>:0: note: loc(fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"]): called from
<unknown>:0: note: loc(callsite(callsite(fused["TensorListReserve:", "functional_1_1/custom_attention_model_10_1/rnn_12_1/TensorArrayV2_1@__inference___call___1940569"] at fused["StatefulPartitionedCall:", "StatefulPartitionedCall@__inference_signature_wrapper___call___1940604"]) at fused["StatefulPartitionedCall:", "StatefulPartitionedCall_1"])): see current operation: %13 = "tf.TensorListReserve"(%6, %3) {device = ""} : (tens